<a href="https://colab.research.google.com/github/saheedniyi02/fpl-ai/blob/master/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
forward_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

leak_columns = [
    "name",
    "team",
]  # columns that shouldnt be used in training fir fear of data leakage


dropped_columns = [
    "season",
    "opponent",
    "match_result",
   # "position",
    "assists",
    "penalties_missed",
    "bonus",
    "bps",
    "clean_sheets",
    "creativity",
    "goals_conceded",
    "goals_scored",
    "ict_index",
    "influence",
    "own_goals",
    "penalties_saved",
    "red_cards",
    "saves",
    "selected",
    "threat",
    "transfers_balance",
    "transfers_in",
    "transfers_out",
    "yellow_cards",
    "team Goal scored",
    "team Goal conceded"
]  # "value",

midfielder_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

goalkeeper_statistics = ['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'total_points_ex', 'minutes_ex', 'goals_conceded_ex',
       'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'red_cards_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3', 'mean goals_conceded 3',
       'mean ict_index 3',  'mean minutes 3',
       'mean own_goals 3',  'mean penalties_saved 3',
        'mean saves 3',  'mean threat 3',
       'mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

statistics =['value', 'position','was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'yellow_cards_ex','now_cost_ex', 'GW', 'opponent_last_season_position', 'mean assists 3',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3',
       'mean creativity 3', 'mean goals_conceded 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3',
       'mean own_goals 3',
       'mean red_cards 3',  'mean threat 3','mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3','mean saves 3','mean assists all',
       'mean bonus all', 'mean bps all', 'mean clean_sheets all',
       'mean creativity all', 'mean goals_conceded all', 'mean goals_scored all',
       'mean ict_index all', 'mean influence all', 'mean minutes all',
       'mean own_goals all',
       'mean red_cards all',  'mean threat all','mean total_points all',
       'mean value all', 'mean match_result all',
       'mean team Goal scored 3','mean team Goal scored all','mean team Goal conceded 3','mean team Goal conceded all',"ratio_goal_scored all","ratio_goal_scored 3",
       'opp mean team Goal scored 3','opp mean team Goal conceded 3','opp mean team Goal scored all','opp mean team Goal conceded all',"opp mean match_result all"]


date_cols=["day_of week","month","hour","week"]

In [2]:
def convert_minutes(val):
    """CONVERTS MINUTES TO A CATEGORICAL OUTPUT"""
    if val > 10:
        return 1
    else:
        return 0
gameweek=17

In [3]:
# !pip install catboost
# !pip install hyperopt
# !pip install lightgbm

In [4]:
from hyperopt import tpe,hp,fmin,STATUS_OK,Trials
from hyperopt.pyll.base import scope

In [5]:
import pandas as pd
import numpy as np
import warnings
import os


warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    confusion_matrix,
    accuracy_score,
    f1_score,
)
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,KFold

In [6]:
train = pd.read_csv("cleaned_dataset/cleaned_previous_seasons.csv",index_col=0)#/content/drive/MyDrive/Fplpredict /cleaned_previous_seasons.csv", index_col=0)
old_gameweek_cleaned = []
for i in range(1, gameweek):
    old_gameweek_cleaned.append(pd.read_csv(f"cleaned_dataset/2023-24/GW{i}.csv"))
old_gameweeks = pd.concat(old_gameweek_cleaned)[train.columns]
train = pd.concat([train, old_gameweeks])

# data for current gameweek we want to predict on
test = pd.read_csv(f"cleaned_dataset/2023-24/GW{gameweek}.csv", index_col=0)

In [7]:
#CREATE GAMEWEEK PREDICTIONS FOLDER
path = f"predicted_dataset/GW{gameweek}"

# create new single directory
# check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)

# data from current season but previous gameweeks

Folder predicted_dataset/GW17 already exists


In [151]:
train["position"].value_counts()

MID    36147
DEF    29940
FWD    10930
GK      8368
GKP     1293
Name: position, dtype: int64

In [152]:
train["position"]=train["position"].replace({"GKP":"GK" })

In [153]:
test = test[train.columns]
train_copy = train.copy()
test_copy = test.copy()

train["index"] = train["name"] + train["kickoff_time"].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train= train.set_index("index")
train["date"]=pd.to_datetime(train["kickoff_time"])
train["day_of week"]=train["date"].dt.day_name
train["month"]=train["date"].dt.month
train["hour"]=train["date"].dt.hour
train["week"]=train["date"].dt.week
train.drop(["kickoff_time","date"], axis=1, inplace=True)

test["index"] = test["name"] + test["kickoff_time"].astype("str")
test= test.set_index("index")
test["date"]=pd.to_datetime(test["kickoff_time"])
test["day_of week"]=test["date"].dt.day_name
test["month"]=test["date"].dt.month
test["hour"]=test["date"].dt.hour
test["week"]=test["date"].dt.week
test.drop(["kickoff_time","date"], axis=1, inplace=True)
train["minutes"] = train["minutes"].apply(convert_minutes)

target = train[["minutes", "GW","position" ]]
train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)
# train.drop(dropped_columns, axis=1, inplace=True)
# test.drop(dropped_columns, axis=1, inplace=True)


# Convert categorical columns to numerical
for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name","position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})

# train = train[statistics + leak_columns+date_cols]
# test = test[statistics + leak_columns+date_cols]

x, val, y, y_val = train_test_split(
    train.drop(leak_columns, axis=1),
    target["minutes"],
    test_size=0.1,
    random_state=0,
)

In [154]:
# train.keys
print(train.shape)
print(test.shape)

(86515, 86)
(765, 86)


In [155]:
params={'colsample_bylevel': 0.8070621518153563, 'learning_rate': 0.04765984972709895, 'max_depth': 7, 'reg_lambda': 5, 'scale_pos_weight': 2.5,'subsample': 0.6794390204583894}
model=CatBoostClassifier(**params,cat_features=["position"],random_state=0,early_stopping_rounds=500,use_best_model=True,verbose=500,n_estimators=10000)

In [156]:
model.fit(x, y,eval_set=[(val,y_val)])

0:	learn: 0.6004799	test: 0.6004432	best: 0.6004432 (0)	total: 50.3ms	remaining: 8m 23s
500:	learn: 0.0337160	test: 0.0410267	best: 0.0410136 (491)	total: 23.2s	remaining: 7m 19s
1000:	learn: 0.0269846	test: 0.0405909	best: 0.0405854 (996)	total: 46.5s	remaining: 6m 57s
1500:	learn: 0.0221590	test: 0.0406678	best: 0.0405009 (1142)	total: 1m 9s	remaining: 6m 35s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.04050091097
bestIteration = 1142

Shrink model to first 1143 iterations.


In [157]:
print(confusion_matrix(model.predict(val), y_val))
print(
    f"starting Accuracy score {accuracy_score(model.predict(val), y_val)}"
)

print(
    f" starting f1 score: {f1_score(model.predict(val), y_val)}"
)

feature_importance = pd.DataFrame(
    {"column": x.columns, "imp": model.feature_importances_}
).sort_values(
    "imp", ascending=False
)  #

[[5075   21]
 [ 154 3402]]
starting Accuracy score 0.9797734627831716
 starting f1 score: 0.9749247743229688


[[3863  267]
 [ 975 2862]]
starting Accuracy score 0.8441069411321702
 starting f1 score: 0.8217054263565893

In [158]:
# feature_importance.head(50)

In [159]:
# feature_importance.tail(20)

In [160]:
# y_val

In [161]:
# test_copy["minutes"] = model.predict(test.drop(leak_columns, axis=1))
test_copy["minutes"] = model.predict(test.drop(leak_columns, axis=1))

test_copy[leak_columns + ["minutes"]].to_csv(
    f"minutes.csv"
)

In [162]:
predicted_minutes=model.predict(val)
val_=pd.DataFrame({"ind":val.index,"actul_minutes":y_val,"predicted_minutes":predicted_minutes,"position":val["position"]})

In [163]:
print(f"STARTING GOALKEEPERS PERFORMANCE!!!")
a=val_[val_["position"]=="GK"]["actul_minutes"]
b=val_[val_["position"]=="GK"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING GOALKEEPERS PERFORMANCE!!!
[[681   0]
 [  0 272]]
starting Accuracy score 1.0
 starting f1 score: 1.0


STARTING GOALKEEPERS PERFORMANCE!!!
[[583  21]
 [ 24 216]]
starting Accuracy score 0.9466824644549763
 starting f1 score: 0.9056603773584907

STARTING DEFENDERS PERFORMANCE!!!
[[1449  208]
 [ 200  915]]
starting Accuracy score 0.8528138528138528
 starting f1 score: 0.817694369973190

STARTING MIDFIELDERS PERFORMANCE!!!
[[1595  288]
 [ 235 1195]]
starting Accuracy score 0.8421370359191066
 starting f1 score: 0.820460006865774

STARTING FORWARDS PERFORMANCE!!!
[[497  88]
 [ 69 325]]
starting Accuracy score 0.839632277834525
 starting f1 score: 0.8054522924411399

In [164]:
print(f"STARTING DEFENDERS PERFORMANCE!!!")
a=val_[val_["position"]=="DEF"]["actul_minutes"]
b=val_[val_["position"]=="DEF"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING DEFENDERS PERFORMANCE!!!
[[1749   36]
 [   7 1154]]
starting Accuracy score 0.9854039375424304
 starting f1 score: 0.981709910676308


In [165]:
print(f"STARTING MIDFIELDERS PERFORMANCE!!!")
a=val_[val_["position"]=="MID"]["actul_minutes"]
b=val_[val_["position"]=="MID"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING MIDFIELDERS PERFORMANCE!!!
[[2018   89]
 [  10 1532]]
starting Accuracy score 0.9728692792545903
 starting f1 score: 0.9687006006955422


In [166]:
print(f"STARTING FORWARDS PERFORMANCE!!!")
a=val_[val_["position"]=="FWD"]["actul_minutes"]
b=val_[val_["position"]=="FWD"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING FORWARDS PERFORMANCE!!!
[[627  29]
 [  4 444]]
starting Accuracy score 0.970108695652174
 starting f1 score: 0.9641693811074918


In [167]:
test_copy[test_copy["minutes"]==1]["position"].value_counts()

Series([], Name: position, dtype: int64)

In [168]:
train = train_copy[train_copy["minutes"] > 0]
test = test_copy[test_copy["minutes"] > 0]

print(train.shape)
print(test.shape)

(36606, 85)
(0, 85)


In [124]:

# predict points
train["index"] = train["name"] + train["kickoff_time"].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train= train.set_index("index")
train["date"]=pd.to_datetime(train["kickoff_time"])
train["day_of week"]=train["date"].dt.day_name
train["month"]=train["date"].dt.month
train["hour"]=train["date"].dt.hour
train["week"]=train["date"].dt.week
train.drop(["kickoff_time","date"], axis=1, inplace=True)

test["index"] = test["name"] + test["kickoff_time"].astype("str")
test = test.set_index("index")
test["date"]=pd.to_datetime(test["kickoff_time"])
test["day_of week"]=test["date"].dt.day_name
test["month"]=test["date"].dt.month
test["hour"]=test["date"].dt.hour
test["week"]=test["date"].dt.week
test.drop(["kickoff_time","date"], axis=1, inplace=True)

target = train[["total_points", "GW","position" ]]
train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)
# train.drop(dropped_columns, axis=1, inplace=True)
# test.drop(dropped_columns, axis=1, inplace=True)


for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name","position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["position"]=train["position"].astype("category")
test["position"]=test["position"].astype("category")
train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})

# train = train[statistics + leak_columns+date_cols]
# test= test[statistics + leak_columns+date_cols]

x, val, y, y_val = train_test_split(
    train.drop(leak_columns, axis=1),
    target["total_points"],
    test_size=0.1,
    random_state=0,
)

In [125]:
from sklearn.model_selection import KFold

#cross_validator to splite the data into folds
folds=KFold(n_splits=8,shuffle=True,random_state=0)

#a dataframe to store the predictions made by each fold
predictions_df=pd.DataFrame()

#list to save the mean absolute errors from validatingon each folds
rmse_val=[]
rmse_X=[]

#a simple catboost regressor
model=LGBMRegressor(**{'colsample_bytree': 0.4199299182268318, 'learning_rate': 0.0032874466037521254, 'max_depth': 9, 'min_split_gain': 0.5685369160138952, 'num_leaves': 99, 'reg_alpha': 0.5621526419488447, 'reg_lambda': 0, 'subsample': 0.6534153111773866}, verbose=-50,random_state=0,early_stopping_rounds=200,n_estimators=10000)

#train model, make predictions and check the validation accuracy on  each fold
for i,(train_index,test_index) in enumerate(folds.split(train.drop(leak_columns, axis=1),target["total_points"])):
    train_fold=train.drop(leak_columns, axis=1).iloc[train_index]
    val_fold=train.drop(leak_columns, axis=1).iloc[test_index]
    y_fold=target["total_points"].iloc[train_index]
    y_val_fold=target["total_points"].iloc[test_index]


    model.fit(train_fold,y_fold,eval_set=[(val_fold,y_val_fold)])
    print(i+1)
    print(train.shape)
    print(test.shape)
    prediction=model.predict(test.drop(leak_columns, axis=1))
    predictions_df[i]=prediction
    rmse_val.append(mean_squared_error(model.predict(val_fold),y_val_fold,squared=False))
    rmse_X.append(mean_squared_error(model.predict(train_fold),y_fold,squared=False))
print(rmse_val)
print(rmse_X)

1
(36261, 86)
(275, 86)
2
(36261, 86)
(275, 86)
3
(36261, 86)
(275, 86)
4
(36261, 86)
(275, 86)
5
(36261, 86)
(275, 86)
6
(36261, 86)
(275, 86)
7
(36261, 86)
(275, 86)
8
(36261, 86)
(275, 86)
[0.3075120598428029, 0.3313043213964431, 0.30399343628970693, 0.313598482155166, 0.3159639171947073, 0.33103955276947383, 0.3199626145714969, 0.3338618037311383]
[0.23496718929023303, 0.23540772150492076, 0.23558363216513445, 0.23580305068105972, 0.23533406006021343, 0.2357738480763343, 0.23440760341313693, 0.2370243783905846]


In [126]:
print(np.mean(rmse_val))
print(np.mean(rmse_X))
#2.7575293874474336
#2.0174134721607295

0.3196545234938669
0.23553768544770215


2.7320675301713573
1.6917027774360753

In [127]:
predictions_df[0]

0      9.144303
1      1.747099
2      2.071056
3      4.962048
4      2.384626
         ...   
270    1.008705
271    1.193375
272    4.900014
273    8.039949
274    4.640778
Name: 0, Length: 275, dtype: float64

In [128]:
test["points"]=np.mean(predictions_df, axis=1).values

test[leak_columns + ["points", "value"]].sort_values(
    "points", ascending=False
).to_csv("points.csv")

In [129]:
test[test["position"]=="MID"].sort_values(by="points",ascending=False).head(11)[["name","points","team"]]

,name,points,team
index,,,
Son Heung-min2023-12-10T16:30:00Z,Son Heung-min,16.672894,Spurs
Richarlison de Andrade2023-12-10T16:30:00Z,Richarlison de Andrade,14.671592,Spurs
Mohamed Salah2023-12-09T12:30:00Z,Mohamed Salah,12.596883,Liverpool
Jack Grealish2023-12-10T14:00:00Z,Jack Grealish,9.933753,Man City
Simon Adingra2023-12-09T15:00:00Z,Simon Adingra,9.806450,Brighton
Abdoulaye Doucouré2023-12-10T14:00:00Z,Abdoulaye Doucouré,9.606363,Everton
Harry Wilson2023-12-10T14:00:00Z,Harry Wilson,9.341238,Fulham
John McGinn2023-12-09T17:30:00Z,John McGinn,9.206036,Aston Villa
Bernardo Veiga de Carvalho e Silva2023-12-10T14:00:00Z,Bernardo Veiga de Carvalho e Silva,9.117368,Man City


In [131]:
test[test["position"]=="DEF"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Marcos Senesi2023-12-09T15:00:00Z,Marcos Senesi,14.826416,Bournemouth
Tosin Adarabioyo2023-12-10T14:00:00Z,Tosin Adarabioyo,13.643207,Fulham
Destiny Udogie2023-12-10T16:30:00Z,Destiny Udogie,8.990474,Spurs
Anel Ahmedhodžić2023-12-09T15:00:00Z,Anel Ahmedhodžić,8.009787,Sheffield Utd
Vitalii Mykolenko2023-12-10T14:00:00Z,Vitalii Mykolenko,6.841697,Everton
Neco Williams2023-12-09T15:00:00Z,Neco Williams,6.557610,Nott'm Forest
Harry Toffolo2023-12-09T15:00:00Z,Harry Toffolo,6.255920,Nott'm Forest
Jack Robinson2023-12-09T15:00:00Z,Jack Robinson,6.254764,Sheffield Utd
Calvin Bassey2023-12-10T14:00:00Z,Calvin Bassey,6.234043,Fulham


In [132]:
test[test["position"]=="GKP"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Jordan Pickford2023-12-10T14:00:00Z,Jordan Pickford,8.790408,Everton
Emiliano Martínez Romero2023-12-09T17:30:00Z,Emiliano Martínez Romero,8.736957,Aston Villa
Bernd Leno2023-12-10T14:00:00Z,Bernd Leno,8.699020,Fulham
Wes Foderingham2023-12-09T15:00:00Z,Wes Foderingham,8.203410,Sheffield Utd
James Trafford2023-12-09T15:00:00Z,James Trafford,6.854552,Burnley
Norberto Murara Neto2023-12-09T15:00:00Z,Norberto Murara Neto,5.602009,Bournemouth
Mark Flekken2023-12-09T15:00:00Z,Mark Flekken,2.811411,Brentford
Robert Sánchez2023-12-10T14:00:00Z,Robert Sánchez,2.767346,Chelsea
Alisson Ramses Becker2023-12-09T12:30:00Z,Alisson Ramses Becker,2.711842,Liverpool


In [133]:
test[test["position"]=="FWD"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Matheus Santos Carneiro Da Cunha2023-12-09T15:00:00Z,Matheus Santos Carneiro Da Cunha,9.907372,Wolves
Raúl Jiménez2023-12-10T14:00:00Z,Raúl Jiménez,9.688809,Fulham
Jean-Philippe Mateta2023-12-09T12:30:00Z,Jean-Philippe Mateta,7.550750,Crystal Palace
Dominic Solanke2023-12-09T15:00:00Z,Dominic Solanke,7.094123,Bournemouth
Elijah Adebayo2023-12-10T14:00:00Z,Elijah Adebayo,6.735416,Luton
Carlos Vinícius Alves Morais2023-12-10T14:00:00Z,Carlos Vinícius Alves Morais,5.008009,Fulham
Julián Álvarez2023-12-10T14:00:00Z,Julián Álvarez,4.989116,Man City
Dominic Calvert-Lewin2023-12-10T14:00:00Z,Dominic Calvert-Lewin,4.861040,Everton
Jay Rodriguez2023-12-09T15:00:00Z,Jay Rodriguez,4.832229,Burnley


In [134]:
test["points"].sort_values(ascending=False).head(50)

index
Son Heung-min2023-12-10T16:30:00Z                          16.672894
Marcos Senesi2023-12-09T15:00:00Z                          14.826416
Richarlison de Andrade2023-12-10T16:30:00Z                 14.671592
Tosin Adarabioyo2023-12-10T14:00:00Z                       13.643207
Mohamed Salah2023-12-09T12:30:00Z                          12.596883
Jack Grealish2023-12-10T14:00:00Z                           9.933753
Matheus Santos Carneiro Da Cunha2023-12-09T15:00:00Z        9.907372
Simon Adingra2023-12-09T15:00:00Z                           9.806450
Raúl Jiménez2023-12-10T14:00:00Z                            9.688809
Abdoulaye Doucouré2023-12-10T14:00:00Z                      9.606363
Harry Wilson2023-12-10T14:00:00Z                            9.341238
John McGinn2023-12-09T17:30:00Z                             9.206036
Bernardo Veiga de Carvalho e Silva2023-12-10T14:00:00Z      9.117368
Destiny Udogie2023-12-10T16:30:00Z                          8.990474
Jordan Pickford2023-12-10T14

In [135]:

# feature_importance = pd.DataFrame(
#     {"column": x.columns, "imp": model.feature_importances_}
# ).sort_values(
#     "imp", ascending=False
# )  #

In [136]:
# feature_importance.head(50)

,column,imp
3,bps,27272
9,influence,23604
5,creativity,14407
8,ict_index,13905
6,goals_conceded,11820
16,threat,8077
0,position,7707
7,goals_scored,7506
1,assists,7051
14,saves,6725


In [137]:
# feature_importance.tail(30)

,column,imp
78,std transfers_out 3,818
29,total_points_ex,768
72,std influence 3,767
77,std transfers_in 3,748
37,ict_index_ex,744
52,mean goals_conceded 3,724
19,transfers_out,677
83,week,673
30,minutes_ex,651
40,yellow_cards_ex,598


#Save predictions

In [143]:
test_copy[test_copy["position"]=="DEF"][["name","team","minutes"]].to_csv(f"predicted_dataset/GW{gameweek}/defenders_minutes.csv")
test_copy[test_copy["position"]=="GKP"][["name","team","minutes"]].to_csv(f"predicted_dataset/GW{gameweek}/goalkeepers_minutes.csv")
test_copy[test_copy["position"]=="MID"][["name","team","minutes"]].to_csv(f"predicted_dataset/GW{gameweek}/midfielders_minutes.csv")
test_copy[test_copy["position"]=="FWD"][["name","team","minutes"]].to_csv(f"predicted_dataset/GW{gameweek}/forwards_minutes.csv")

In [144]:
test[test["position"]=="DEF"][["name","team","points","value"]].sort_values(by="points",ascending=False).to_csv(f"predicted_dataset/GW{gameweek}/defenders_points.csv")
test[test["position"]=="GKP"][["name","team","points","value"]].sort_values(by="points",ascending=False).to_csv(f"predicted_dataset/GW{gameweek}/goalkeepers_points.csv")
test[test["position"]=="MID"][["name","team","points","value"]].sort_values(by="points",ascending=False).to_csv(f"predicted_dataset/GW{gameweek}/midfielders_points.csv")
test[test["position"]=="FWD"][["name","team","points","value"]].sort_values(by="points",ascending=False).to_csv(f"predicted_dataset/GW{gameweek}/forwards_points.csv")

In [142]:
test_copy[test_copy["position"]=="DEF"][["name","team","minutes"]]

,name,team,minutes
4,Rúben Gato Alves Dias,Man City,1
7,Rico Lewis,Man City,0
9,João Cancelo,Man City,0
12,Nathan Aké,Man City,1
14,Manuel Akanji,Man City,0
...,...,...,...
664,Max Lowe,Sheffield Utd,1
665,Chris Basham,Sheffield Utd,0
667,Jayden Bogle,Sheffield Utd,1
669,Anel Ahmedhodžić,Sheffield Utd,1
